# species

This is a text cell. Start editing!

In [1]:
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import group.research.aging.spark.extensions.functions.ConcatenateString
import group.research.aging.spark.extensions.functions.Concatenate
import ammonite.ops._
import ammonite.ops.ImplicitWd._

In [2]:
val species = spark.read.parquet("/data/ensembl/99/website/species_lookup.parquet").na.fill("")
val species_n = species.select(species.columns.head,species.columns.tail.toList.dropRight(9):_*).as[(String, String, String, String, String, String, String, String, 
java.lang.Double, java.lang.Double,java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double, 
java.lang.Double, java.lang.Double,java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double)]

species_n.where($"class" === "Mammalia").show(10,10000)

In [3]:
val prefix = "http://aging-research.group/resource/"
val prefix_ensembl = "http://rdf.ebi.ac.uk/resource/ensembl/"
val prefix_samples = "http://aging-research.group/samples/"
def samp(str: String) = "<" +prefix_samples +str.replace(" ", "_")+">"
def pref(str: String) = "<" +prefix +str.replace(" ", "_")+">" 
def ens(str: String) =  "<" +prefix_ensembl +str.replace(" ", "_")+">"
def tax(str: String) = ens("taxon#"+str)
def str(s: String) = if(s==null) "" else s""""${s.replace("\"", "\\\"")}""""

In [4]:
def trip(s: String, p: String, o: String):List[(String, String, String, String)] = if(s==null || p == null || o == null ||
 s == "" ||  p == "" || o == "" || o=="\\N" || o.startsWith("\"\\N\"") || s == "\\N" || s == "\"\\N\"") Nil else List((s,p,o, "<" + "http://rdf.ebi.ac.uk/resource/ensembl/confidence/high" + "> ."))
def trip_sp(s: String, p: String, o: Any):List[(String, String, String, String)] = {
  if(o == null) Nil else  trip(pref(s),pref(p), o.toString) 
}
/*
def sp(taxon_id: java.lang.Integer, scientific_name: String)(p: String, o: Any) = if(taxon_id==null || o == null) Nil else {
      trip(tax(taxon_id.toString), pref(p), o.toString)++
        trip(pref(scientific_name),pref(p), o.toString) 
}*/

## Load species



In [6]:
val species = spark.read.parquet("/data/ensembl/99/website/species_lookup.parquet").na.fill("")
val species_n = species.select(species.columns.head,species.columns.tail.toList.dropRight(9):_*).as[(String, String, String, String, String, String, String, String, 
java.lang.Double, java.lang.Double,java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double, 
java.lang.Double, java.lang.Double,java.lang.Double, java.lang.Double, java.lang.Double, java.lang.Double)]

species_n.where($"class" === "Mammalia").show(10,10000)

+--------+--------+------------+--------------+------------+-------------+-------------------------+-----------------------+----------------+-----------+--------------+-----------+------------------+--------------+--------------------+------------------+-------+-----------+----------------+--------------------+--------------+----------------+
|  phylum|   class|       order|        family|       genus|      species|          scientific_name|            common_name|maximum_lifespan|body_mass_g|metabolic_rate|temperature|temperature_kelvin|gestation_days|female_maturity_days|male_maturity_days|weaning|litter_size|litters_per_year|inter_birth_interval|birth_weight_g|weaning_weight_g|
+--------+--------+------------+--------------+------------+-------------+-------------------------+-----------------------+----------------+-----------+--------------+-----------+------------------+--------------+--------------------+------------------+-------+-----------+----------------+-------------------

In [7]:
val trs = species_n.flatMap{
    case (phylum, animal_class, order, family,   genus, species, scientific_name, common_name, maximum_lifespan, 
    body_mass_g, metabolic_rate, _,  temperature_kelvin, 
    gestation_days, female_maturity_days, male_maturity_days, weaning, litter_size, litters_per_year, 
    inter_birth_interval, birth_weight_g, weaning_weight_g) =>
        //trip(pref(scientific_name),pref("has_taxon"), tax(taxon_id.toString))++
        trip_sp(scientific_name,"has_common_name", str(common_name))++
        trip_sp(scientific_name,"has_lifespan", maximum_lifespan)++
        trip_sp(scientific_name,"has_temperature", Some(temperature_kelvin).map(t=>if(t>274 && !t.isInfinite && !t.isNaN) t - 273.15 else null).getOrElse(null) )++        
        trip_sp(scientific_name,"has_temperature_kelvin", temperature_kelvin)++
        trip_sp(scientific_name,"has_mass_g", body_mass_g) ++
        trip_sp(scientific_name,"has_metabolic_rate", metabolic_rate) ++
        trip_sp(scientific_name,"is_animal_class", ens(animal_class)) ++
        trip_sp(scientific_name,"of_order", ens(order)) ++
        trip_sp(scientific_name,"of_family", ens(family)) ++        
        trip_sp(scientific_name,"has_gestation_days", gestation_days)++
        trip_sp(scientific_name,"has_female_maturity_days", female_maturity_days)++ 
        trip_sp(scientific_name,"has_male_maturity_days", male_maturity_days)++
        trip_sp(scientific_name,"has_weaning", weaning)++
        trip_sp(scientific_name,"has_litter_size", litter_size)++ 
        trip_sp(scientific_name,"has_litters_per_year", litters_per_year)++ 
        trip_sp(scientific_name,"has_inter_birth_interval", inter_birth_interval)++ 
        trip_sp(scientific_name,"has_birth_weight_g", birth_weight_g)++
        trip_sp(scientific_name,"weaning_weight_g", weaning_weight_g)++
        Nil
}.toDF("subject", "property", "object", "context")
trs.show(100,1000)

+--------------------------------------------------------------+---------------------------------------------------------------+------------------------------------------------------+---------------------------------------------------------+
|                                                       subject|                                                       property|                                                object|                                                  context|
+--------------------------------------------------------------+---------------------------------------------------------------+------------------------------------------------------+---------------------------------------------------------+
|      <http://aging-research.group/resource/Daphnia_pulicaria>|         <http://aging-research.group/resource/has_common_name>|                                             "Daphnia"|<http://rdf.ebi.ac.uk/resource/ensembl/confidence/high> .|
|      <http://aging-research.gr

In [8]:
trs.writeTSV("/data/databases/graphdb/import/species_up.nq", false)

parts of /data/databases/graphdb/import/species_up.nq merged!


/data/databases/graphdb/import/species_up.nq